In [24]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

In [25]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt) #A
        for i in range(0, len(token_ids) - max_length, stride): #B
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
        
    def __len__(self): #C
        return len(self.input_ids)
    def __getitem__(self, idx): #D
        return self.input_ids[idx], self.target_ids[idx]

In [26]:
def create_dataloader_v1(txt, batch_size=4,max_length=256, 
                            stride=128, shuffle=True, drop_last=True, 
                            num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2") #A
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) #B
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle,
        drop_last=drop_last, 
        num_workers = num_workers
    )
    return dataloader

In [27]:
with open ("dataset/natural_language/the-verdict.txt", encoding="utf-8", mode='r') as f:
    raw_text = f.read()

In [28]:
len(raw_text)

20479

In [29]:
tokenizer = tiktoken.get_encoding("gpt2")
tokenizer.n_vocab

50257

In [30]:
vocab_size = 50257
output_dim = 256

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [31]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

In [32]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [33]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [34]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


In [35]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
